In [ ]:
import os
import sys
import time
import math
import tqdm
import imageio
import pathlib
import dataclasses
import collections
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datasets import load_dataset
from openpi.training import config as _config
from openpi.policies import policy_config
from openpi_client import image_tools

sys.path.append("/home/wyz/openpi/third_party/libero")
from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TORCH_DISABLE_DYNAMO"] = "1"
os.environ["PYTORCH_JIT"] = "0"

LIBERO_DUMMY_ACTION = [0.0] * 6 + [-1.0]
LIBERO_ENV_RESOLUTION = 256


# ------------------------------------------------------------
# ✅ 加载策略
# ------------------------------------------------------------
def load_policy():
    print("Loading policy checkpoint ...")
    config = _config.get_config("pi05_libero")
    ckpt = "/home/wyz/openpi/checkpoints/pi05_libero_pytorch.pt"

    def _no_compile(x, *args, **kwargs):
        return x
    torch.compile = _no_compile

    try:
        torch.jit._state.disable()
    except Exception:
        pass

    policy = policy_config.create_trained_policy(config, ckpt, pytorch_device="cuda:0")
    print("✅ Policy loaded:", type(policy))
    if hasattr(policy, "model"):
        print(" → Using model:", policy.model.__class__.__module__)
    return policy


# ------------------------------------------------------------
# ✅ 工具函数
# ------------------------------------------------------------
def _quat2axisangle(quat):
    quat = np.array(quat)
    quat = np.clip(quat, -1.0, 1.0)
    den = np.sqrt(max(1e-6, 1.0 - quat[3] ** 2))
    if math.isclose(den, 0.0):
        return np.zeros(3)
    return (quat[:3] * 2.0 * math.acos(quat[3])) / den


def _get_libero_env(task, resolution, seed):
    task_desc = task.language
    bddl_path = pathlib.Path(get_libero_path("bddl_files")) / task.problem_folder / task.bddl_file
    env_args = {"bddl_file_name": bddl_path, "camera_heights": resolution, "camera_widths": resolution}
    env = OffScreenRenderEnv(**env_args)
    env.seed(seed)
    return env, task_desc


# ------------------------------------------------------------
# ✅ SigLIP Attention 可视化函数（改进版）
# ------------------------------------------------------------
def analyze_siglip_attention(attn_weights):
    """
    分析 SigLIP attention weights 的统计特性，判断是否真的均匀。
    
    Args:
        attn_weights: torch.Tensor, shape [Batch, Heads, Tokens, Tokens]
    
    Returns:
        dict: 包含统计信息的字典
    """
    if attn_weights is None:
        return None
    
    if isinstance(attn_weights, torch.Tensor):
        attn_weights = attn_weights.float().numpy()
    
    attn_weights = attn_weights[0]  # [Heads, Tokens, Tokens]
    attn_weights = np.mean(attn_weights, axis=0)  # [Tokens, Tokens]
    
    # 计算统计信息
    stats = {
        'mean': float(np.mean(attn_weights)),
        'std': float(np.std(attn_weights)),
        'min': float(np.min(attn_weights)),
        'max': float(np.max(attn_weights)),
        'entropy': float(-np.sum(attn_weights * np.log(attn_weights + 1e-10))),  # Attention entropy
        'uniformity': float(np.std(attn_weights) / (np.mean(attn_weights) + 1e-10)),  # 均匀度（越小越均匀）
    }
    
    return stats


def process_siglip_attention_weights(attn_weights, patch_size=14, image_size=224, method='rollout'):
    """
    改进的 SigLIP attention weights 处理，支持多种可视化方法。
    
    Args:
        attn_weights: torch.Tensor, shape [Batch, Heads, Tokens, Tokens]
        patch_size: int, SigLIP 的 patch size (默认 14)
        image_size: int, 输入图像大小 (默认 224)
        method: str, 可视化方法:
            - 'mean': 平均所有 token 的 attention（原始方法）
            - 'center': 中心 token 对其他 token 的 attention
            - 'rollout': 聚合所有 token 的 attention（推荐）
            - 'importance': 每个 token 的重要性（所有其他 token 对它的 attention）
    
    Returns:
        heatmap: np.ndarray, shape [H, W] 的 2D heatmap
    """
    if attn_weights is None:
        return None
    
    # 转换为 numpy
    if isinstance(attn_weights, torch.Tensor):
        attn_weights = attn_weights.float().numpy()
    
    # 取第一个 batch
    attn_weights = attn_weights[0]  # [Heads, Tokens, Tokens]
    
    # 平均所有 heads
    attn_weights = np.mean(attn_weights, axis=0)  # [Tokens, Tokens]
    
    # 计算 patch grid 大小
    num_patches_per_side = image_size // patch_size
    num_patches = num_patches_per_side * num_patches_per_side
    
    # 确保只使用 patch tokens（排除可能的 CLS token）
    if attn_weights.shape[0] > num_patches:
        attn_weights = attn_weights[:num_patches, :num_patches]
    
    if method == 'mean':
        # 原始方法：平均所有 token 的 attention
        token_attn = np.mean(attn_weights, axis=0)  # [Tokens]
    elif method == 'center':
        # 中心 token 对其他 token 的 attention
        center_idx = num_patches // 2
        token_attn = attn_weights[center_idx, :]  # [Tokens]
    elif method == 'rollout':
        # Rollout 方法：聚合所有 token 的 attention
        # 对每个 token，计算它接收到的 attention 的总和
        token_attn = np.sum(attn_weights, axis=0)  # 每个 token 作为 key 接收到的 attention
    elif method == 'importance':
        # 重要性方法：每个 token 的重要性（所有其他 token 对它的 attention）
        token_attn = np.sum(attn_weights, axis=0)  # 所有 token 作为 query 对它的 attention
    else:
        raise ValueError(f"Unknown method: {method}. Choose from: 'mean', 'center', 'rollout', 'importance'")
    
    # 重塑为 2D grid
    heatmap = token_attn.reshape(num_patches_per_side, num_patches_per_side)
    
    # 归一化到 [0, 1]
    if heatmap.max() > heatmap.min():
        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
    else:
        # 如果所有值都相同，返回均匀的 heatmap
        heatmap = np.ones_like(heatmap) * 0.5
    
    # 上采样到原始图像大小
    try:
        from scipy import ndimage
        heatmap = ndimage.zoom(heatmap, (image_size / num_patches_per_side, image_size / num_patches_per_side), order=1)
    except ImportError:
        # 如果没有 scipy，使用 torch 的插值
        heatmap_tensor = torch.from_numpy(heatmap).unsqueeze(0).unsqueeze(0)  # [1, 1, H, W]
        heatmap_tensor = torch.nn.functional.interpolate(
            heatmap_tensor, 
            size=(image_size, image_size), 
            mode='bilinear', 
            align_corners=False
        )
        heatmap = heatmap_tensor.squeeze().numpy()
    
    return heatmap


def visualize_siglip_attention_heatmap(image, heatmap, save_path=None, alpha=0.5, cmap='jet'):
    """
    可视化 SigLIP attention heatmap，叠加到原始图像上。
    
    Args:
        image: np.ndarray, 原始图像 [H, W, 3], uint8
        heatmap: np.ndarray, 2D heatmap [H, W]
        save_path: str, 保存路径（可选）
        alpha: float, 叠加透明度
        cmap: str, colormap 名称
    """
    if heatmap is None:
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 原始图像
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Heatmap
    im = axes[1].imshow(heatmap, cmap=cmap, interpolation='bilinear')
    axes[1].set_title('Attention Heatmap')
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1])
    
    # 叠加
    axes[2].imshow(image)
    axes[2].imshow(heatmap, cmap=cmap, alpha=alpha, interpolation='bilinear')
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        pathlib.Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    else:
        plt.show()


# ------------------------------------------------------------
# ✅ 参数类
# ------------------------------------------------------------
@dataclasses.dataclass
class Args:
    task_suite_name: str = "libero_spatial"
    num_trials_per_task: int = 1       # ✅ 每个任务只跑一次
    num_steps_wait: int = 10
    replan_steps: int = 5
    resize_size: int = 224
    video_out_path: str = "./libero_videos_prune"
    seed: int = 7
    # SigLIP Attention 可视化选项
    visualize_siglip_attention: bool = True
    siglip_attention_save_dir: str = "./siglip_attention_heatmaps"
    siglip_attention_save_freq: int = 10  # 每 N 步保存一次
    siglip_attention_method: str = "rollout"  # 可视化方法: 'mean', 'center', 'rollout', 'importance'


# ------------------------------------------------------------
# ✅ 主评估函数
# ------------------------------------------------------------
def eval_libero_prune(args: Args):
    policy = load_policy()
    np.random.seed(args.seed)

    # ----------------------
    print("🔥 Warming up policy (compile + autotune)...")
    dummy_element = {
        "observation/image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/wrist_image": np.zeros((args.resize_size, args.resize_size, 3), dtype=np.uint8),
        "observation/state": np.zeros(8, dtype=np.float32),
        "prompt": "dummy task",
    }
    with torch.inference_mode():
        _ = policy.infer(dummy_element)
    torch.cuda.synchronize()
    print("✅ Warmup done, start evaluation...\n")

    # ----------------------
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite = benchmark_dict[args.task_suite_name]()
    num_tasks = task_suite.n_tasks

    pathlib.Path(args.video_out_path).mkdir(parents=True, exist_ok=True)
    
    # 创建 SigLIP attention 保存目录
    if args.visualize_siglip_attention:
        pathlib.Path(args.siglip_attention_save_dir).mkdir(parents=True, exist_ok=True)

    # 最大步数设定
    if args.task_suite_name == "libero_spatial":
        max_steps = 220
    elif args.task_suite_name == "libero_object":
        max_steps = 280
    elif args.task_suite_name == "libero_goal":
        max_steps = 300
    else:
        max_steps = 400

    total_episodes, total_successes = 0, 0
    total_infer_times = []

    print(f"\n🔹 Running evaluation on suite: {args.task_suite_name} ({num_tasks} tasks)")

    for task_id in tqdm.tqdm(range(num_tasks), desc="Tasks"):
        task = task_suite.get_task(task_id)
        init_states = task_suite.get_task_init_states(task_id)
        init_states = init_states[:args.num_trials_per_task]

        env, desc = _get_libero_env(task, LIBERO_ENV_RESOLUTION, args.seed)
        task_episodes, task_successes = 0, 0
        task_infer_times = []

        for ep in range(args.num_trials_per_task):
            obs = env.set_init_state(init_states[ep])
            env.reset()
            action_plan = collections.deque()
            replay_imgs = []

            # 环境稳定等待
            for _ in range(args.num_steps_wait):
                obs, _, _, _ = env.step(LIBERO_DUMMY_ACTION)

            done = False
            t = 0
            while t < max_steps + args.num_steps_wait:
                try:
                    img = np.ascontiguousarray(obs["agentview_image"][::-1, ::-1])
                    wrist = np.ascontiguousarray(obs["robot0_eye_in_hand_image"][::-1, ::-1])

                    img = image_tools.convert_to_uint8(image_tools.resize_with_pad(img, args.resize_size, args.resize_size))
                    wrist = image_tools.convert_to_uint8(image_tools.resize_with_pad(wrist, args.resize_size, args.resize_size))
                    replay_imgs.append(img)

                    if not action_plan:
                        element = {
                            "observation/image": img,
                            "observation/wrist_image": wrist,
                            "observation/state": np.concatenate(
                                (obs["robot0_eef_pos"], _quat2axisangle(obs["robot0_eef_quat"]), obs["robot0_gripper_qpos"])
                            ),
                            "prompt": str(desc),
                        }

                        # ✅ 纯推理时间测量
                        with torch.inference_mode():
                            torch.cuda.synchronize()
                            t0 = time.perf_counter()
                            actions = policy.infer(element)["actions"]
                            torch.cuda.synchronize()
                            infer_time = (time.perf_counter() - t0) * 1000
                            task_infer_times.append(infer_time)

                        # ✅ SigLIP Attention 可视化（多层）
                        if args.visualize_siglip_attention and t % args.siglip_attention_save_freq == 0:
                            if hasattr(policy, '_model') and hasattr(policy._model, 'last_siglip_attention_maps'):
                                attention_maps_by_layer = policy._model.last_siglip_attention_maps
                                if attention_maps_by_layer is not None and len(attention_maps_by_layer) > 0:
                                    # 处理每一层的 attention
                                    for layer_idx, attention_maps in attention_maps_by_layer.items():
                                        if len(attention_maps) > 0:
                                            # 处理第一个图像的 attention（agentview）
                                            attn_weights = attention_maps[0]  # 第一个图像
                                            
                                            # 分析 attention 统计特性
                                            stats = analyze_siglip_attention(attn_weights)
                                            if stats:
                                                print(f"  📊 Layer {layer_idx} SigLIP Attention Stats: std={stats['std']:.4f}, uniformity={stats['uniformity']:.4f}, entropy={stats['entropy']:.2f}")
                                            
                                            # 生成 heatmap
                                            heatmap = process_siglip_attention_weights(
                                                attn_weights, 
                                                patch_size=14,  # SigLIP So400m/14 使用 14x14 patch
                                                image_size=args.resize_size,
                                                method=args.siglip_attention_method
                                            )
                                            if heatmap is not None:
                                                save_path = pathlib.Path(args.siglip_attention_save_dir) / f"task{task_id}_ep{ep}_step{t}_layer{layer_idx}_siglip_{args.siglip_attention_method}_agentview.png"
                                                visualize_siglip_attention_heatmap(img, heatmap, save_path=str(save_path), alpha=0.5)
                                                print(f"  💾 Saved Layer {layer_idx} SigLIP attention heatmap ({args.siglip_attention_method}): {save_path}")

                        action_plan.extend(actions[: args.replan_steps])

                    action = action_plan.popleft()
                    obs, reward, done, info = env.step(action.tolist())

                    if done:
                        task_successes += 1
                        total_successes += 1
                        break

                    t += 1

                except Exception as e:
                    print(f"[⚠️] Exception in task {task_id}, ep {ep}: {e}")
                    break

            task_episodes += 1
            total_episodes += 1

            tag = "success" if done else "failure"
            seg = desc.replace(" ", "_")
            imageio.mimwrite(
                pathlib.Path(args.video_out_path) / f"rollout_{seg}_{tag}_ep{ep}.mp4",
                [np.asarray(x) for x in replay_imgs],
                fps=10,
            )

        # ===== 每个任务结果 =====
        sr = 100 * task_successes / max(task_episodes, 1)
        total_sr = 100 * total_successes / max(total_episodes, 1)
        avg_infer = np.mean(task_infer_times)
        total_infer_times.extend(task_infer_times)

        print(f"✅ Task {task_id}: {desc}")
        print(f" Success rate: {task_successes}/{task_episodes} = {sr:.1f}%")
        print(f" Avg inference time: {avg_infer:.2f} ms")
        print(f" Overall success: {total_successes}/{total_episodes} = {total_sr:.1f}%\n")

    # ===== 汇总结果 =====
    print("=" * 60)
    print(f"🎯 Final success rate on {args.task_suite_name}: {total_successes}/{total_episodes} = {(100 * total_successes / total_episodes):.2f}%")
    print(f"⚙️ Avg inference time over all tasks: {np.mean(total_infer_times):.2f} ± {np.std(total_infer_times):.2f} ms")
    print("=" * 60)


# ------------------------------------------------------------
# ✅ 运行入口
# ------------------------------------------------------------
if __name__ == "__main__":
    args = Args(task_suite_name="libero_spatial", num_trials_per_task=1)  # ✅ 每个任务只跑一次
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        eval_libero_prune(args)



/home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /home/wyz/miniconda3/envs/pi0/lib/python3.11/site-packages/robosuite/scripts/setup_macros.py (__init__.py:9)
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.

Loading policy checkpoint ...
✅ Policy loaded: <class 'openpi.policies.policy.Policy'>
🔥 Warming up policy (compile + autotune)...
✅ Warmup done, start evaluation...

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

🔹 Running evaluation on suite: libero_spatial (10 tasks)


Tasks:   0%|          | 0/10 [00:00<?, ?it/s]

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
  📊 Layer 0 SigLIP Attention Stats: std=0.0152, uniformity=3.8814, entropy=1028.97
  💾 Saved Layer 0 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task0_ep0_step0_layer0_siglip_rollout_agentview.png
  📊 Layer 26 SigLIP Attention Stats: std=0.0093, uniformity=2.3843, entropy=1178.36
  💾 Saved Layer 26 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task0_ep0_step0_layer26_siglip_rollout_agentview.png
  📊 Layer 0 SigLIP Attention Stats: std=0.0151, uniformity=3.8775, entropy=1029.01
  💾 Saved Layer 0 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task0_ep0_step10_layer0_siglip_rollout_agentview.png
  📊 Layer 26 SigLIP Attention Stats: std=0.0090, uniformity=2.2952, entropy=1176.61
  💾 Saved Layer 26 SigLIP atten

Tasks:  10%|█         | 1/10 [00:08<01:18,  8.76s/it]

...

✅ Task 7: pick up the black bowl on the stove and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 124.73 ms
 Overall success: 8/8 = 100.0%

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
  📊 Layer 0 SigLIP Attention Stats: std=0.0151, uniformity=3.8734, entropy=1029.89
  💾 Saved Layer 0 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task8_ep0_step0_layer0_siglip_rollout_agentview.png
  📊 Layer 26 SigLIP Attention Stats: std=0.0095, uniformity=2.4336, entropy=1177.50
  💾 Saved Layer 26 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task8_ep0_step0_layer26_siglip_rollout_agentview.png
  📊 Layer 0 SigLIP Attention Stats: std=0.0151, uniformity=3.8748, entropy=1029.36
  💾 Saved Layer 0 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task8

Tasks:  90%|█████████ | 9/10 [01:32<00:11, 11.35s/it]

  💾 Saved Layer 26 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task8_ep0_step130_layer26_siglip_rollout_agentview.png
✅ Task 8: pick up the black bowl next to the plate and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 126.09 ms
 Overall success: 9/9 = 100.0%

[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /home/wyz/Isaac-GR00T/external_dependencies/LIBERO/libero/libero/../datasets does not exist!
  📊 Layer 0 SigLIP Attention Stats: std=0.0150, uniformity=3.8348, entropy=1036.58
  💾 Saved Layer 0 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task9_ep0_step0_layer0_siglip_rollout_agentview.png
  📊 Layer 26 SigLIP Attention Stats: std=0.0103, uniformity=2.6244, entropy=1160.13
  💾 Saved Layer 26 SigLIP attention heatmap (rollout): siglip_attention_heatmaps/task9_ep0_step0_layer26_siglip_rollout_agentview.png
  📊 Layer 0 SigLIP Attentio

Tasks: 100%|██████████| 10/10 [01:43<00:00, 10.38s/it]

✅ Task 9: pick up the black bowl on the wooden cabinet and place it on the plate
 Success rate: 1/1 = 100.0%
 Avg inference time: 125.70 ms
 Overall success: 10/10 = 100.0%

🎯 Final success rate on libero_spatial: 10/10 = 100.00%
⚙️ Avg inference time over all tasks: 125.25 ± 3.59 ms
